In [ ]:
def mcmc_sample(hops,pdf, pdfparam):                 
    """
    Samples points for a given function with markov chain monte carlo (mcmc)
    Parameters
    ----------
    pdf: function
        function to be sampled
    n_samples: int
        number of sampled points
    pdfparam: list
        parameters that the pdf takes (if normal, standard deviation, mean...)    
        
    possible improvement: make number of walkers a controllable parameter.
    """
    states = []
    n_accepted = 0
    r_rp = np.zeros(hops)
    rp_r = np.zeros(hops)
    current = random.uniform(-4,4)                      # generates numbers around zero in a uniform way, only for the first position of the "walker"
    for i in range(hops):                               # hops is the number of times you "hop", i.e. nr of steps...
        states.append(current)
        movement = current + random.uniform(-hop_size,hop_size)
        if i % n_hopsperwalker == 0 and i!=0:
            current = random.uniform(-4,4)
        curr_prob = pdf(current, *pdfparam)
        move_prob = pdf(movement, *pdfparam)            # same prob from 1 to 2 than from 2 to 1, easier in cartesian. Book by jos uses spherical -> would need some scaling 
                                                        # to account for different volume in spherical shells of different radius. Probably it would be a factor r/r'
        
        acceptance = min(move_prob/curr_prob,1)         # acceptance is A_RR'
        invacceptance = min(curr_prob/move_prob,1)
        r_rp[i] = acceptance * curr_prob
        rp_r[i] = invacceptance * move_prob
        if random_coin(acceptance) and i % n_hopsperwalker != 0:
            current = movement
            n_accepted += 1
            #print("accepted", i, "n_acc", n_accepted)
    #print ("accepted/total =", n_accepted/hops)
    
    #error
    b = 1
    N = int(hops-burn_in)
    R = states[burn_in:]
    S_a = np.zeros((N, 1))
    for b in range(1, N):
        Nb = int(N/b)
        r=np.zeros(Nb)
        R=R[:(Nb*b)]
        R_shape=np.reshape(R, (Nb,b))
        R = states[burn_in:]
        r = np.sum(R_shape, axis = 1)/b
        S_a[b]=np.sqrt((1/(Nb-1))*(((1/Nb)*np.sum(np.square(r)))-np.square((1/Nb)*np.sum(r))))

    return states[burn_in:], n_accepted/hops, r_rp[burn_in:], rp_r[burn_in:]                            # give the system some time to find a reasonable starting point. 